## Load data

In [1]:
import json

input_path = './data/covid-bbn.json'
output_path = './network/covid.R'

with open(input_path, 'r') as fp:
    d = json.load(fp)

In [2]:
d.keys()

dict_keys(['nodes', 'edges', 'parameters'])

## Util

In [3]:
from pathlib import Path

def delete_file(path):
    p = Path(path)
    if p.exists():
        p.unlink()
    
def save(path, s):
    with open(path, 'a') as fp:
        fp.write(s)
        fp.write('\n\n')

In [4]:
delete_file(output_path)

## Libraries

In [5]:
_s = [
    'library(bnlearn)',
    'library(gRain)'
]
_s = '\n'.join(_s)

save(output_path, _s)

## Graph

### Nodes

In [6]:
_s = ','.join([f"'{k}'" for k in d['nodes'].keys()])
_s = f'c({_s})'
_s = f'g <- empty.graph(nodes={_s})'

save(output_path, _s)

### Edges

In [7]:
_s = [f'g <- set.arc(g, from="{d["parent"]}", to="{d["child"]}")' for d in d['edges']]
_s = '\n'.join(_s)

save(output_path, _s)

## Parameters

### Values

In [8]:
def get_values(node_key):
    values = [f'"{v}"' for v in d['nodes'][node_key]['values'].keys()]
    values = ', '.join(values)
    values = f'c({values})'
    values = f'{node_key}.lv <- {values}'
    return values

_s = [get_values(k) for k in d['nodes'].keys()]
_s = '\n'.join(_s)

save(output_path, _s)

### Probabilities

In [9]:
def get_dim(node_key):
    if len(d['nodes'][node_key]['values']) == len(d['parameters'][node_key]):
        s = len(d['nodes'][node_key]['values'])
    else:
        s = [k for k in d['parameters'][node_key][0] if k != '__p__']
        s = [len(d['nodes'][k]['values']) for k in s]
        s = [f'{k}' for k in s]
        s = ', '.join(s)
        s = f'c({s})'
    
    return s

def get_dimnames(node_key):
    r = d['parameters'][node_key][0]
    r = [k for k in r if k not in {'__p__', node_key}]
    r = [f'{node_key}={node_key}.lv'] + [f'{k}={k}.lv' for k in r][::-1]
    r = ', '.join(r)
    r = f'list({r})'
    return r

def get_p(node_key):
    r = d['parameters'][node_key]
    r = [_r['__p__'] for _r in r]
    r = [f'{_r}' for _r in r]
    r = ', '.join(r)
    r = f'c({r})'
    
    return r

def get_prob(node_key):
    dim = get_dim(node_key)
    dimnames = get_dimnames(node_key)
    p = get_p(node_key)
    
    return f'{node_key}.prob <- array({p}, dim={dim}, dimnames={dimnames})'

_s = [get_prob(k) for k in d['nodes'].keys()]
_s = '\n'.join(_s)

save(output_path, _s)

In [10]:
_s = [f'{k}={k}.prob' for k in d['nodes'].keys()]
_s = ', '.join(_s)
_s = f'p <- list({_s})'

save(output_path, _s)

## BBN + Tree

In [11]:
_s = [
    'b <- custom.fit(g, p)',
    't <- compile(as.grain(b))'
]
_s = '\n'.join(_s)

save(output_path, _s)